In [171]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split


#  House pricing analysis 

Cieľ: na základe dát ako ulica na ktorej sa byt nachádza, vybavenie bytu, susedstvo, atď. **odhadnite cenu bytu**.

Dátový súbor obsahuje 2919 záznamom a 81 premenných (po spojení tréningového a testovacieho súboru, z dôvodu rovnakej úpravy dát. Pred predikciou bol opäť rozdelený).

Zvolený postup: 
1. zoznámenie sa s dátami (predchádzajúci notebook)
2. vysporiadanie sa s chýbajúcimi pozorovaniami:
- odstránenie niektorých záznamov (ak chábalo jediné porozovanie z celého datasetu)
- nahradenie "missing values" (numerical) pomocou mediánu (nie je najlepšia technika. Chcel som použiť nahradenie pomocou KNN algorytmu, avšak mal som problém s načítaním knižnice)
- ponechanie "missing values" z dôvodu, že by aj oni mohli priniesť informáciu pre odhad ceny
3. "One hot encoding" kategorických premenných 
4. Zvolenie správnej Neurónovej siete pre odhad ceny
5. Finálny odhad

In [172]:
df1 = pd.read_csv('C:/Users/zamec/Desktop/M7DataSp/cvičenie 1/house-prices-advanced-regression-techniques/HousepricesAll.csv')

In [173]:
df1.shape

(2919, 81)

Zistenie počtu chábajúcich pozorovaní pre numerické premenné. 
- Ak sa v celom stĺpci nachádzal len jeden chábajúci záznam, tak som vymazal len daný riadok. Takto som prišiel o 3 záznamy.
- V prípade premennej MaSVnArea so tiež odstránil 23 zázánamov.
- Premenné LotFrontageArea, GarageYrBlt - som chábajúce pozorovania nahradil mediánom z daného stĺpcu

In [174]:
integerDrop = []

for i in range(df1.shape[1]):
    if (df1.dtypes[i] == 'int64' or df1.dtypes[i] == 'float64') and df1.isna().sum()[i]:
        
        print("In Column %s is %d missing values" %(df1.columns[i],df1.isna().sum()[i]))
        
        if df1.isna().sum()[i] < 24:
            integerDrop.append(df1.columns[i])

In Column LotFrontage is 486 missing values
In Column MasVnrArea is 23 missing values
In Column BsmtFinSF1 is 1 missing values
In Column BsmtFinSF2 is 1 missing values
In Column BsmtUnfSF is 1 missing values
In Column TotalBsmtSF is 1 missing values
In Column BsmtFullBath is 2 missing values
In Column BsmtHalfBath is 2 missing values
In Column GarageYrBlt is 159 missing values
In Column GarageCars is 1 missing values
In Column GarageArea is 1 missing values


In [175]:
df1 = df1.dropna(axis = 0,subset = integerDrop)
df1.shape

(2893, 81)

Premenné ktoré neobsahovali viac ako 90% záznamov (niektoré až 99%).

In [176]:
df1.drop(["PoolQC","Fence","MiscFeature","Alley","FireplaceQu"], axis = 1, inplace = True)


.

Skontrolovanie počtu katogórií kategorických premenných a uvažovnie nad možnosťou spojiť tie, ktoré obsahujú malý počet pozorovaní do kategórie **others**. Nakoniec som kategórie ponechal v pôvodnom tvare.

In [177]:
categoricalColumns = []
for name in df1.columns:
    
    if df1.dtypes[name] == "O":
        categoricalColumns.append(name)
        print( "The column %s has: %d categories" %(name,len(df1[name].unique())))

The column MSZoning has: 6 categories
The column Street has: 2 categories
The column LotShape has: 4 categories
The column LandContour has: 4 categories
The column Utilities has: 3 categories
The column LotConfig has: 5 categories
The column LandSlope has: 3 categories
The column Neighborhood has: 25 categories
The column Condition1 has: 9 categories
The column Condition2 has: 8 categories
The column BldgType has: 5 categories
The column HouseStyle has: 8 categories
The column RoofStyle has: 6 categories
The column RoofMatl has: 8 categories
The column Exterior1st has: 16 categories
The column Exterior2nd has: 17 categories
The column MasVnrType has: 5 categories
The column ExterQual has: 4 categories
The column ExterCond has: 5 categories
The column Foundation has: 6 categories
The column BsmtQual has: 5 categories
The column BsmtCond has: 5 categories
The column BsmtExposure has: 5 categories
The column BsmtFinType1 has: 7 categories
The column BsmtFinType2 has: 7 categories
The colu

In [178]:
medians = df1.loc[:,['LotFrontage','GarageYrBlt']].median()

df1['LotFrontage'] = df1['LotFrontage'].replace(np.nan,medians[0])
df1['GarageYrBlt'] = df1['GarageYrBlt'].replace(np.nan,medians[1])

df1 = df1.replace(np.nan,'None')

Kategorizácia kategorických premenných pre potreby NN.

In [179]:
df1 = pd.get_dummies(df1, columns=categoricalColumns)

In [187]:
#split na tie co budem odhadovat a testovacie a treningove 
dfPrediction = df1.loc[df1['SalePrice'] == 0 ]
dfTestTrain = df1.loc[df1['SalePrice'] != 0 ]

labels = dfTestTrain['SalePrice'].array
dfTestTrain.drop(['SalePrice'], axis = 1, inplace = True)

C:\Users\zamec\Anaconda3\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [191]:
labels = np.array(labels)

In [192]:
X_train, X_test, y_train, y_test = train_test_split(dfTestTrain, labels, train_size = 0.8, random_state = 20)


print("Train shape is: (%d,%d)" %(X_train.shape[0],X_train.shape[1]))
print("Test shape is: (%d,%d)" %(X_test.shape[0],X_test.shape[1]))
print("Train target shape is: (%d,)" %(y_train.shape[0]))
print("Train target shape is: (%d,)" %(y_test.shape[0]))

Train shape is: (1161,288)
Test shape is: (291,288)
Train target shape is: (1161,)
Train target shape is: (291,)


Po bližšom nahliadnutí na rôznorodosť variability dát (min, max, mean), som sa rozhodol pre škálovanie dát pomocou **MinmaxScalet**, čo je typická technika pri danom probléme. Pomocou neho môžme výrazne pomôcť k rýchlejšej konvergenicii neurénovej siete.

In [193]:
X_train.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,SaleType_New,SaleType_None,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
count,1161.000000,1161.000000,1161.000000,1161.000000,1161.000000,1161.000000,1161.000000,1161.000000,1161.000000,1161.000000,...,1161.000000,1161.0,1161.000000,1161.000000,1161.000000,1161.000000,1161.000000,1161.000000,1161.000000,1161.000000
mean,730.233419,57.734711,69.619294,10507.368648,6.091301,5.587425,1970.985357,1985.050818,104.810508,439.101637,...,0.079242,0.0,0.002584,0.874246,0.072351,0.003445,0.009475,0.009475,0.823428,0.081826
std,422.298740,42.973953,22.688933,10732.347731,1.388475,1.136911,30.321592,20.711150,182.101218,456.458050,...,0.270233,0.0,0.050789,0.331714,0.259181,0.058621,0.096917,0.096917,0.381470,0.274218
min,1.000000,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,365.000000,20.000000,60.000000,7446.000000,5.000000,5.000000,1953.000000,1967.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
50%,730.000000,50.000000,68.000000,9350.000000,6.000000,5.000000,1972.000000,1994.000000,0.000000,384.000000,...,0.000000,0.0,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
75%,1098.000000,70.000000,78.000000,11478.000000,7.000000,6.000000,2000.000000,2004.000000,168.000000,704.000000,...,0.000000,0.0,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
max,1460.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,...,1.000000,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [194]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)


## Výber architektúry neurónovej siete

Nakoniec som zvolil architektúru typu **288-64-64-32-1** aj z dôvodu aby nebola príliš robustná a NN sa rýchlo nepreučila. 
- Stálo by zváženie skúsenie rôznych druhov aktivačných funkcii ako **sigmoid, tanh...** avšak k najlepšej MAE (priemernej absolútnej chybe) som sa dostal pomocou nasledovnej NN.

In [195]:
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D,BatchNormalization
from keras import optimizers



model = Sequential()
model.add(Dense(64, activation='relu',input_shape=(X_train.shape[1],)))
model.add(Dense(64, activation='selu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])

model.summary()

Model: "sequential_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_79 (Dense)             (None, 64)                18496     
_________________________________________________________________
dense_80 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_81 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_82 (Dense)             (None, 1)                 33        
Total params: 24,769
Trainable params: 24,769
Non-trainable params: 0
_________________________________________________________________


.


Najnižšie s MAE som sa dostal k hodnote **15268** , tj. priemerný rozdiel medzi reálnou a odhadnoutou cenou testovacieho súboru bolo 15000 dolárov. To je stále vysoký rozdiel a určite by sa táto hodnota dala ešte znížiť použitím odlišnej NN.

In [196]:
models_data = model.fit(X_train, y_train, 
          batch_size=1, epochs=150, verbose=0,validation_data=(X_test,y_test))

In [163]:
model.save(r"C:\Users\zamec\Desktop\M7DataSp\cvičenie 1\FirstModel.h5")

In [201]:
model = load_model(r"C:\Users\zamec\Desktop\M7DataSp\cvičenie 1\FirstModel.h5")

In [202]:
dfPrediction.head()


,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,SaleType_None,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,SalePrice
1460,1461,20,80.0,11622,5,6,1961,1961,0.0,468.0,...,0,0,1,0,0,0,0,1,0,115372.742188
1461,1462,20,81.0,14267,6,6,1958,1958,108.0,923.0,...,0,0,1,0,0,0,0,1,0,100577.031250
1462,1463,60,74.0,13830,5,5,1997,1998,0.0,791.0,...,0,0,1,0,0,0,0,1,0,189002.937500
1463,1464,60,78.0,9978,6,6,1998,1998,20.0,602.0,...,0,0,1,0,0,0,0,1,0,201688.890625
1464,1465,120,43.0,5005,8,5,1992,1992,0.0,263.0,...,0,0,1,0,0,0,0,1,0,189872.453125


In [203]:
dfPrediction.drop(['SalePrice'], axis = 1, inplace = True)

C:\Users\zamec\Anaconda3\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [204]:
dfPrediction1 = scaler.transform(dfPrediction)


In [205]:
SalePrice = []

for i in range(dfPrediction.shape[0]):
    
    x = np.expand_dims(dfPrediction1[i,:], axis=0)
    
    SalePrice.append(model.predict(x)[0][0])
    
dfPrediction['SalePrice'] = SalePrice    



C:\Users\zamec\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


**Finálny súbor s predikovanými hodntami zo súboru test.csv**

In [206]:
dfPrediction

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,SaleType_None,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,SalePrice
1460,1461,20,80.0,11622,5,6,1961,1961,0.0,468.0,...,0,0,1,0,0,0,0,1,0,122324.062500
1461,1462,20,81.0,14267,6,6,1958,1958,108.0,923.0,...,0,0,1,0,0,0,0,1,0,100377.789062
1462,1463,60,74.0,13830,5,5,1997,1998,0.0,791.0,...,0,0,1,0,0,0,0,1,0,191525.187500
1463,1464,60,78.0,9978,6,6,1998,1998,20.0,602.0,...,0,0,1,0,0,0,0,1,0,203431.796875
1464,1465,120,43.0,5005,8,5,1992,1992,0.0,263.0,...,0,0,1,0,0,0,0,1,0,187347.437500
1465,1466,60,75.0,10000,6,5,1993,1994,0.0,0.0,...,0,0,1,0,0,0,0,1,0,177151.640625
1466,1467,20,68.0,7980,6,7,1992,2007,0.0,935.0,...,0,0,1,0,0,0,0,1,0,185451.781250
1467,1468,60,63.0,8402,6,5,1998,1998,0.0,0.0,...,0,0,1,0,0,0,0,1,0,169553.843750
1468,1469,20,85.0,10176,7,5,1990,1990,0.0,637.0,...,0,0,1,0,0,0,0,1,0,188047.375000
1469,1470,20,70.0,8400,4,5,1970,1970,0.0,804.0,...,0,0,1,0,0,0,0,1,0,129595.726562
